# Day 3 — Baselines & Metrics

**Goals:**
- Train logistic baseline
- Confusion matrix/ROC/PR
- Calibrate probability threshold

In [ ]:
import sys, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
rng = np.random.default_rng(0)
print('Python:', sys.version.split()[0])